In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from bayesmixpy import run_mcmc

In [ ]:
import os
os.environ["BAYESMIX_EXE"] = '../../build/run_mcmc'

## Generate data


In [ ]:
data = np.concatenate([
    np.random.normal(loc=3, scale=1, size=100),
    np.random.normal(loc=-3, scale=1, size=100),
])
plt.hist(data)
plt.show()

## The Bayesian Model

\begin{align*}
    y_i \mid \theta_i=(\mu_i, \sigma^2_i) & \sim \mathcal{N}(\mu_i, \sigma^2_i) \\
    \theta_i \mid P & \sim P \\
    P & \sim DP(\alpha G_0) 
\end{align*}
And $G_0(d\mu, d\sigma^2) = \mathcal N(d\mu | \mu_0, \sigma^2/\lambda) \mathcal{IG}(\sigma^2 | a, b)$

We consider different prior specifications

### Fixed hyperparameters

$\alpha = 1$

$(\mu_0, \lambda, a, b) = (0, 0.1, 2, 2)$


In [ ]:
dp_params_fix = """
fixed_value {
    totalmass: 1.0
}
"""

g0_params_fix = """
fixed_values {
    mean: 0.0
    var_scaling: 0.1
    shape: 2.0
    scale: 2.0
}
"""

### Prior on $\alpha$ and $\mu_0$

$\alpha \sim \text{Gamma}(2, 2)$

$\mu_0 \sim \mathcal{N}(0, 10)$

$(\lambda, a, b) = (0.1, 2, 2)$

In [ ]:
dp_params_prior = """
gamma_prior {
  totalmass_prior {
    shape: 4.0
    rate: 2.0
  }
}
"""

g0_params_meanprior = """
normal_mean_prior {
    mean_prior {
        mean: 0.0
        var: 10.0
    }
    var_scaling: 0.1
    shape: 2.0
    scale: 2.0
}
"""

dp_params = [dp_params_fix, dp_params_prior]

### Prior on all the hyperparameters

$\alpha \sim \text{Gamma}(2, 2)$

$\mu_0 \sim \mathcal{N}(0, 10)$

$\lambda \sim \text{Gamma}(0.2, 0.6)$

$a = 1.5$

$b \sim \text{Gamma}(4, 2)$

In [ ]:
g0_params_allprior = """
ngg_prior {
    mean_prior {
        mean: 5.5
        var: 2.25
    }
    var_scaling_prior {
        shape: 0.2
        rate: 0.6
    }
    shape: 1.5
    scale_prior {
        shape: 4.0
        rate: 2.0
    }
}
"""

g0_params = [g0_params_fix, g0_params_meanprior, g0_params_allprior]

## The algorithm

We consider all available algorithms in bayesmix: Neal's Algorithms 2, 3 and 8

In [ ]:
neal2_algo = """
algo_id: "Neal2"
rng_seed: 20201124
iterations: 10
burnin: 5
init_num_clusters: 3
"""

neal3_algo = """
algo_id: "Neal3"
rng_seed: 20201124
iterations: 10
burnin: 5
init_num_clusters: 3
"""

neal8_algo = """
algo_id: "Neal8"
rng_seed: 20201124
iterations: 10
burnin: 5
init_num_clusters: 3
neal8_n_aux: 3
"""

algorithms = [neal2_algo, neal3_algo, neal8_algo]
algo_names = ["Neal2", "Neal3", "Neal8"]

### We're interested in the predictive density

`return_clusters=False, return_num_clusters=False, return_best_clus=False`

Observe that the number of iterations is extremely small! In real problems, you might want to set the burnin at least to 1000 iterations and the total number of iterations to at least 2000.

In [ ]:
fix, axes = plt.subplots(2, 3, figsize=(12, 6))

dens_grid = np.linspace(-10, 10, 1000)

for i, dp in enumerate(dp_params):
    for j, g0 in enumerate(g0_params):
        for k, algo in enumerate(algorithms):
            eval_dens = run_mcmc(
                "NNIG", "DP", data, g0, dp, algo, dens_grid,
                return_clusters=False, return_num_clusters=False,
                return_best_clus=False)[0]
            
            axes[i, j].plot(dens_grid, np.exp(np.mean(eval_dens, axis=0)),
                            label="Algo: {0}".format(algo_names[k]))

axes[0, 0].set_ylabel("DP - fix")
axes[1, 0].set_ylabel("DP - gamma prior")

axes[0, 0].set_title("G0 - fix")
axes[0, 1].set_title("G0 - mean prior")
axes[0, 2].set_title("G0 - NGG prior")

axes[1, 2].legend(ncol=3, bbox_to_anchor=(0.4, -0.2), fontsize=16)
plt.show()

## What about the clustering ?

We can extract 
1. The full chain of the cluster allocations
2. The chain of the number of clusters
3. The "best" cluster according to Binder loss function

In [ ]:
neal2_algo = """
algo_id: "Neal2"
rng_seed: 20201124
iterations: 2000
burnin: 1000
init_num_clusters: 3
"""

In [ ]:
_ , numcluschain, cluschain, bestclus = run_mcmc(
    "NNIG", "DP", data, g0_params_allprior, dp_params_prior, neal2_algo, 
    dens_grid=None, return_clusters=True, return_num_clusters=True,
    return_best_clus=True)

In [ ]:
x, y = np.unique(numcluschain, return_counts=True)
plt.bar(x, y / y.sum())
plt.xticks(x)
plt.title("Posterior distribution of the number of clusters")
plt.show()

In [ ]:
plt.hist(data, alpha=0.3, density=True)
for c in np.unique(bestclus):
    data_in_clus = data[bestclus == c]
    plt.scatter(data_in_clus, np.zeros_like(data_in_clus) + 0.01)
plt.show()